# Backtesting

In [1]:
import pandas as pd
import numpy as np
import math
import yfinance as yf
from tqdm.notebook import tqdm_notebook
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_cola = yf.download('KO', start='2022-04-08',
               end='2023-01-01', auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [2]:
for i in [20, 60]:
    df_cola[f"MA_{i}"] = df_cola["Close"].rolling(i).mean()
df_cola = df_cola.dropna()
df_cola.sample(3)

,Open,High,Low,Close,Volume,MA_20,MA_60
Date,,,,,,,
2022-12-27,61.009567,61.353123,60.799616,61.276775,7320700,60.589664,57.184135
2022-09-30,53.889876,54.136253,53.065467,53.084419,16124600,56.349503,58.463536
2022-08-31,58.683302,58.824411,58.043599,58.053005,14633600,60.002210,59.050832


In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.01, row_heights=[0.7, 0.3])
fig.add_trace(go.Candlestick(x=df_cola.index,
            open=df_cola['Open'], high=df_cola['High'],
            low=df_cola['Low'], close=df_cola['Close'],
            showlegend=True, name='Close Price'),row=1,col=1 )
fig.add_scatter(x=df_cola.index, y=df_cola['MA_20'], name='MA20')               
fig.add_scatter(x=df_cola.index, y=df_cola['MA_60'], name='MA60') 
fig.add_trace(go.Bar(x=df_cola.index, y=df_cola.Volume,
                     showlegend=True,name='Volume',
                    marker=dict(color='rgb(125,125,222)')),row=2,col=1)
fig.update_layout(title="CocaCola Share Price (Close) US$",
                  xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
def indicator(row): #fastMA, slowMA
    if row.MA_20 > row.MA_60 :
        signal = 1
    elif row.MA_20 < row.MA_60 :
        signal = -1
    else: 
        signal = 0
    return signal

In [ ]:
df_cola[['MA_20', 'MA_60']].apply(indicator, axis=1)

Date
2022-07-06   -1
2022-07-07   -1
2022-07-08   -1
2022-07-11   -1
2022-07-12   -1
             ..
2022-12-23    1
2022-12-27    1
2022-12-28    1
2022-12-29    1
2022-12-30    1
Length: 125, dtype: int64

In [ ]:
df_i = pd.DataFrame({"Indicator": df_cola[['MA_20', 'MA_60']].apply(indicator, axis=1) })
np.where((df_i.Indicator==1)&(df_i.Indicator.shift(1)==-1), "B", 
        np.where((df_i.Indicator==-1)&(df_i.Indicator.shift(1)==1), "S", "Hold"))

array(['Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'B', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'S', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'B', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 

In [ ]:
df_i['Action'] = np.where((df_i.Indicator==1)&(df_i.Indicator.shift(1)==-1), "B", 
            np.where((df_i.Indicator==-1)&(df_i.Indicator.shift(1)==1), "S", "Hold"))

In [ ]:
df_i 

,Indicator,Action
Date,,
2022-07-06,-1,Hold
2022-07-07,-1,Hold
2022-07-08,-1,Hold
2022-07-11,-1,Hold
2022-07-12,-1,Hold
...,...,...
2022-12-23,1,Hold
2022-12-27,1,Hold
2022-12-28,1,Hold


In [ ]:
df_i.loc[df_i['Action']=='B']

,Indicator,Action
Date,,
2022-07-26,1,B
2022-11-15,1,B


In [ ]:
df_i.loc[df_i['Action']=='S']

,Indicator,Action
Date,,
2022-09-14,-1,S


## Simplify our code

In [11]:
fastma = f'MA_20'
slowma = f'MA_60'

def signal(df, fastma, slowma):
    """based on given df and 2 MA, return buy:1/sell:-1/hold:0 signal"""
    return np.where((df[fastma]>df[slowma])&(df[fastma].shift(1)<df[slowma].shift(1)), 1, 
            np.where((df[fastma]<df[slowma])&(df[fastma].shift(1)>df[slowma].shift(1)), -1, 0))

In [12]:
signal(df_cola, fastma, slowma).tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [13]:
df_strategy = df_cola[['Close']]

In [14]:
df_strategy["Signal"] = signal(df_cola, fastma, slowma).tolist()

/var/folders/k0/qltbxq952h32hdmn8lyxcsnh0000gn/T/ipykernel_79649/2630632433.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Avoid adding a series or list without same index, because it might confuse pd.DF  (like the above)

In [15]:
df_strategy.loc[df_strategy.index, ["Signal"]] = signal(df_cola, fastma, slowma).tolist()

In [16]:
df_strategy.loc[df_strategy.index,['Cash']] = np.NaN
df_strategy.loc[df_strategy.index,['Stock']] = np.NaN

In [17]:
df_strategy

,Close,Signal,Cash,Stock
Date,,,,
2022-07-06,59.652256,0,NaN,NaN
2022-07-07,59.181885,0,NaN,NaN
2022-07-08,59.398258,0,NaN,NaN
2022-07-11,59.210106,0,NaN,NaN
2022-07-12,58.956108,0,NaN,NaN
...,...,...,...,...
2022-12-23,60.904583,0,NaN,NaN
2022-12-27,61.276775,0,NaN,NaN
2022-12-28,60.666008,0,NaN,NaN


In [18]:
# buy signal
df_strategy.loc[df_strategy['Signal']==1]

,Close,Signal,Cash,Stock
Date,,,,
2022-07-26,59.464104,1,NaN,NaN
2022-11-15,57.452850,1,NaN,NaN


In [19]:
# sell signal
df_strategy.loc[df_strategy['Signal']==-1]

,Close,Signal,Cash,Stock
Date,,,,
2022-09-14,57.187519,-1,NaN,NaN


## Try not to use for loop directly to deal with DF

#### Use df.iterrows, df.itertuples, df.items instead

Pandas Iteration: 
https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#iteration

List Comprehensions: 
https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions

In [20]:
def backtest(df, cash=1.0, comm_rate = 0.0, stock = 0.0, rf_rate = 0.01):
    """Backtest given strategy DF"""
    """row[0] is Close price; row[1] is Signal; """
    """Signal=1 means buy; Signal=-1 means sell """
    cash_init = cash
    trading_times = 0
    for i, row in df.iterrows():
        # buy signal
        if row['Signal']==1:
            stock = (cash * (1 - comm_rate)) / row[0]
            comm = cash / row[0] * comm_rate
            cash = 0
            df.at[i,'Cash'] = cash
            df.at[i,'Stock'] = stock
            trading_times += 1 
            print(f"{i}> Bought {stock:.4f}share @${row[0]:.4f}, Comm:{comm:.2f}, Cash Bal.:{cash:.4f}")

        # sell signal
        elif row['Signal']==-1:
            cash = stock * row[0] * (1 - comm_rate)
            comm = stock * row[0] * comm_rate
            print(f"{i}> Sold {stock:.4f}share @${row[0]:.4f}, Comm:{comm:.2f}, Cash Bal.:{cash:.4f}")
            stock = 0
            df.at[i,'Cash'] = cash
            df.at[i,'Stock'] = stock
            trading_times += 1
        
        # hold
        else:
            df.at[i,'Cash'] = cash
            df.at[i,'Stock'] = stock
        
    # force cash out at the end term if still held stock
    if stock > 0:
        cash = stock * df.iat[-1,0] * (1 - comm_rate)
        comm = stock * df.iat[-1,0] * comm_rate
        print(f"Force selling @${df.iat[-1,0]:.4f} at end term. Cash Bal.:{cash:.4f}")
        stock = 0
        df.at[i,'Cash'] = cash
        df.at[i,'Stock'] = stock
        trading_times += 1
    
    # summary output
    simple_rtn = (cash - cash_init)/cash_init
    holding_days = df[df['Stock']>0].shape[0]
    total_days = df.shape[0]
    df.loc[df.index, ['Amount']] = [x[2] + x[3]*x[0] for x in np.array(df)]
    df.loc[df.index, ['Daily_rtn']] = df['Amount'].pct_change()
    std = df_strategy['Daily_rtn'].std()
    sharpe = (simple_rtn - rf_rate) / std / np.sqrt(total_days) * np.sqrt(250) # annualized sharpe ratio
    print(f"Sharpe:{sharpe:.4f}, Retrun:{simple_rtn*100:.4}%, SD:{std:.5f}")
    print(f"Transaction times:{trading_times}, hold {holding_days}days, total period:{total_days}days")
    
    return [sharpe, simple_rtn, std, trading_times, holding_days, total_days]

backtest(df_strategy)        

2022-07-26 00:00:00> Bought 0.0168share @$59.4641, Comm:0.00, Cash Bal.:0.0000
2022-09-14 00:00:00> Sold 0.0168share @$57.1875, Comm:0.00, Cash Bal.:0.9617
2022-11-15 00:00:00> Bought 0.0167share @$57.4529, Comm:0.00, Cash Bal.:0.0000
Force selling @$60.7042 at end term. Cash Bal.:1.0161
Sharpe:1.2349, Retrun:1.614%, SD:0.00703
Transaction times:4, hold 66days, total period:125days


[1.2349363177692723, 0.01613967425288143, 0.007030978417301726, 4, 66, 125]

In [21]:
df_strategy.sample(5)

,Close,Signal,Cash,Stock,Amount,Daily_rtn
Date,,,,,,
2022-07-19,58.824406,0,1.0,0.000000,1.000000,0.000000
2022-12-02,61.410378,0,0.0,0.016739,1.027961,0.008779
2022-08-25,60.837585,0,0.0,0.016817,1.023098,0.004817
2022-12-23,60.904583,0,0.0,0.016739,1.019494,0.007578
2022-08-12,59.925068,0,0.0,0.016817,1.007752,0.007593


## Create MA test combination pairs

In [22]:
def pairup(min_MA=20, max_MA=100, interv=10):
    counter = 0
    fast_MA, slow_MA = min_MA, min_MA+interv # initiate combination MAs
    comb_list = []

    for f in tqdm_notebook(range(min_MA, max_MA-interv+1)):
        for s in range(f+interv, max_MA+1):
            comb_list.append([f, s])
            slow_MA += 1
            counter += 1
        
        fast_MA += 1
    
    print(f"Total combinations: {counter}")
    return comb_list

comb_list = pairup()

  0%|          | 0/71 [00:00<?, ?it/s]

Total combinations: 2556


In [23]:
comb_list[0:7]

[[20, 30], [20, 31], [20, 32], [20, 33], [20, 34], [20, 35], [20, 36]]

In [24]:
comb_list[-7:]

[[87, 100], [88, 98], [88, 99], [88, 100], [89, 99], [89, 100], [90, 100]]

In [25]:
len(comb_list)

2556

## Prepare a larger DF with all MA

In [26]:
df_ko = yf.download('KO', start='2003-01-01',
               end='2023-01-01', auto_adjust=True)

for i in range(20, 101):
    df_ko[f"MA_{i}"] = df_ko["Close"].rolling(i).mean()
    
df_ko = df_ko.dropna()
df_ko.sample(3)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,MA_20,MA_21,MA_22,MA_23,MA_24,...,MA_91,MA_92,MA_93,MA_94,MA_95,MA_96,MA_97,MA_98,MA_99,MA_100
Date,,,,,,,,,,,,,,,,,,,,,
2017-06-22,36.181364,36.269475,36.069224,36.101265,7446300,36.219334,36.198167,36.155812,36.109886,36.058518,...,34.350913,34.324897,34.305113,34.292201,34.278566,34.262426,34.246457,34.229688,34.212145,34.197393
2008-10-03,16.451214,16.824411,16.026016,16.081078,27695800,16.172575,16.153553,16.133223,16.114001,16.100177,...,16.218417,16.234298,16.249967,16.264147,16.272883,16.282819,16.293889,16.303347,16.312431,16.319915
2008-05-19,17.229827,17.453121,17.181547,17.356562,20009200,17.522227,17.555002,17.581506,17.613184,17.648131,...,17.919969,17.932356,17.943638,17.950024,17.955896,17.959647,17.964187,17.971387,17.978533,17.987664


In [27]:
df_ko.shape

(4936, 86)

In [28]:
df_ko.describe()

,Open,High,Low,Close,Volume,MA_20,MA_21,MA_22,MA_23,MA_24,...,MA_91,MA_92,MA_93,MA_94,MA_95,MA_96,MA_97,MA_98,MA_99,MA_100
count,4936.000000,4936.000000,4936.000000,4936.000000,4.936000e+03,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000,...,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000,4936.000000
mean,27.957202,28.153553,27.758360,27.960209,1.562449e+07,27.865975,27.860967,27.855950,27.850928,27.845916,...,27.531672,27.527047,27.522417,27.517778,27.513132,27.508479,27.503818,27.499151,27.494481,27.489808
std,13.468117,13.564186,13.367369,13.467261,7.577843e+06,13.394287,13.390555,13.386828,13.383109,13.379427,...,13.193267,13.190649,13.188009,13.185340,13.182646,13.179927,13.177185,13.174420,13.171639,13.168842
min,10.665819,10.742897,10.541958,10.638304,2.147400e+06,10.937076,10.940412,10.938941,10.941905,10.944967,...,10.983609,10.996034,11.005993,11.018085,11.030006,11.038189,11.045825,11.053466,11.062194,11.070470
25%,15.494502,15.602976,15.352101,15.494780,1.069970e+07,15.450606,15.450881,15.447407,15.441620,15.444922,...,15.084081,15.077536,15.064131,15.064286,15.053087,15.050487,15.040784,15.030603,15.030113,15.022876
50%,27.086037,27.213772,26.886472,27.044760,1.385460e+07,27.077160,27.082081,27.091571,27.097649,27.116202,...,26.505686,26.496672,26.488155,26.477993,26.470723,26.457628,26.445713,26.437555,26.430455,26.424370
75%,36.699362,36.898260,36.509779,36.673983,1.833900e+07,36.727266,36.702955,36.708392,36.718966,36.703226,...,36.445849,36.445475,36.446145,36.450823,36.452484,36.449929,36.451787,36.448689,36.445733,36.445473
max,62.577396,62.764192,61.381885,61.839539,1.241690e+08,60.731360,60.733428,60.708562,60.709411,60.698903,...,59.427053,59.441194,59.452120,59.461123,59.462563,59.464168,59.463198,59.461047,59.456517,59.451300


## Modify our code to prepare large data test

In [29]:
# Utilities functions

def back_test(df, cash=1.0, comm_rate = 0.0, stock = 0.0, rf_rate = 0.01):
    """Backtest given strategic DF"""
    """Signal=1 means buy; Signal=-1 means sell """
    """row[0] is Close price; row[1] is Signal; """
    trading_times = 0
    for i, row in df.iterrows():
        # buy signal
        if (row['Signal']==1) and (cash>0):
            stock = (cash * (1 - comm_rate)) / row[0]
            comm = cash * comm_rate
            cash = 0
            df.at[i,'Cash'] = cash
            df.at[i,'Stock'] = stock
            df.at[i,'Comm'] = comm
            trading_times += 1
            #print(f"{i}> Bought {stock:.4f}share @${row[0]:.4f}, Comm:{comm:.2f}, Cash Bal.:{cash:.4f}")

        # sell signal
        elif (row['Signal']==-1) and (stock>0) :  
            cash = stock * row['Close'] * (1 - comm_rate)
            comm = stock * row['Close'] * comm_rate
            #print(f"{i}> Sold {stock:.4f}share @${row[0]:.4f}, Comm:{comm:.2f}, Cash Bal.:{cash:.4f}")
            stock = 0
            df.at[i,'Cash'] = cash
            df.at[i,'Stock'] = stock
            df.at[i,'Comm'] = comm
            trading_times += 1
        
        # hold
        else:
            df.at[i,'Cash'] = cash
            df.at[i,'Stock'] = stock
        
    # force cash out at the end term if still held stock
    if stock > 0:
        cash = stock * df.iat[-1,0] * (1 - comm_rate)
        comm = stock * df.iat[-1,0] * comm_rate
        #print(f"Force selling @${df.iat[-1,0]:.4f} at end term. Cash Bal.:{cash:.4f}")
        stock = 0
        df.at[i,'Cash'] = cash
        df.at[i,'Stock'] = stock
        df.at[i,'Comm'] = comm
        trading_times += 1
    
    # summary output
    holding_days = df[df['Stock']>0].shape[0]
    total_days = df.shape[0]
    df.loc[df.index, ['Amount']] = [x[2] + x[3]*x[0] for x in np.array(df)]
    df.loc[df.index, ['Daily_rtn']] = np.log(df['Amount']/df['Amount'].shift(1)) # daily log return
    std = df['Daily_rtn'].tail(250).std()
    comm = df['Comm'].sum()
    simple_rtn = df['Amount'][-1]/df['Amount'][0] - 1 
    sharpe = ((df['Amount'][-1]/df['Amount'][-251]-1-rf_rate)/np.sqrt(250))/std  # twailing 250 days' sharpe ratio
    
    return [sharpe, simple_rtn, std, trading_times, holding_days, total_days, comm]

def signal(df, fma, sma):
    """based on given df and 2 MA, return buy:1/sell:-1/hold:0 signal"""
    """Usage: signal(df, 20, 40) """
    return np.where((df[f"MA_{fma}"]>df[f"MA_{sma}"])&(df[f"MA_{fma}"].shift(1)<df[f"MA_{sma}"].shift(1)), 1, 
        np.where((df[f"MA_{fma}"]<df[f"MA_{sma}"])&(df[f"MA_{fma}"].shift(1)>df[f"MA_{sma}"].shift(1)), -1, 0))

In [30]:
# create df to store each comparason data
result = pd.DataFrame(columns=['Pairs','Sharpe','Simple_rtn','STD','Trading_times',
                               'Holding_days','Total_days','Comm'])
result = result.set_index(['Pairs'])

for c in tqdm_notebook(comb_list):
    # create df for temp input signal data for comparason each time
    try:
        del df_strategic
    except:
        pass
    df_strategic = df_ko[['Close']]    
    
    # run each pair of test and build buy/sell/hold signal
    df_strategic.loc[df_strategic.index, ["Signal"]] = signal(df_ko, c[0], c[1]).tolist()
    
    # set record for Cash and Stock
    df_strategic.loc[df_strategic.index,['Cash']] = np.NaN
    df_strategic.loc[df_strategic.index,['Stock']] = np.NaN
    df_strategic.loc[df_strategic.index,['Comm']] = 0
    
    # run test and get result
    ma_test = back_test(df_strategic)
    
    # keep result of sharpe, simple_rtn, std, trading_times, holding_days, total_days
    result.loc[f"MA_{c[0]} MA_{c[1]}"] = ma_test

result.sample(5)

  0%|          | 0/2556 [00:00<?, ?it/s]

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_27 MA_66,-0.173628,1.148271,0.010468,74.0,3177.0,4936.0,0.0
MA_47 MA_91,-0.462210,0.900010,0.010805,58.0,3138.0,4936.0,0.0
MA_74 MA_99,0.066938,1.844369,0.010190,56.0,3274.0,4936.0,0.0
MA_56 MA_88,-0.197209,1.632688,0.010537,66.0,3146.0,4936.0,0.0
MA_31 MA_90,-0.233709,1.001300,0.010488,64.0,3149.0,4936.0,0.0


# While test completed, you would see the following


In [31]:
result.to_csv("result.csv")

In [32]:
result

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_20 MA_30,-0.129045,0.012500,0.009751,218.0,2917.0,4936.0,0.0
MA_20 MA_31,-0.158229,0.155261,0.009755,202.0,2935.0,4936.0,0.0
MA_20 MA_32,0.137919,0.099921,0.009951,190.0,2955.0,4936.0,0.0
MA_20 MA_33,0.211508,0.194131,0.010014,194.0,2967.0,4936.0,0.0
MA_20 MA_34,0.232057,0.286108,0.010045,176.0,2978.0,4936.0,0.0
...,...,...,...,...,...,...,...
MA_88 MA_99,0.560491,1.060843,0.010458,74.0,3290.0,4936.0,0.0
MA_88 MA_100,0.415510,1.143894,0.010353,68.0,3285.0,4936.0,0.0
MA_89 MA_99,0.400712,1.100510,0.010354,78.0,3295.0,4936.0,0.0


In [33]:
result_bak = result

In [34]:
result.nlargest(5, 'Simple_rtn')

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_74 MA_85,0.286326,2.510839,0.010384,100.0,3158.0,4936.0,0.0
MA_72 MA_85,0.506194,2.440181,0.010326,84.0,3148.0,4936.0,0.0
MA_74 MA_86,0.246520,2.430510,0.010381,94.0,3157.0,4936.0,0.0
MA_73 MA_86,0.241862,2.425157,0.010398,84.0,3148.0,4936.0,0.0
MA_73 MA_87,0.324279,2.391282,0.010387,78.0,3143.0,4936.0,0.0


In [35]:
result.nlargest(5, 'Sharpe')

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_85 MA_98,1.199657,1.644792,0.010513,72.0,3272.0,4936.0,0.0
MA_86 MA_98,1.085775,1.702331,0.010561,74.0,3278.0,4936.0,0.0
MA_86 MA_97,1.085775,1.869398,0.010561,82.0,3279.0,4936.0,0.0
MA_85 MA_97,1.085374,1.980373,0.010565,74.0,3273.0,4936.0,0.0
MA_85 MA_96,1.068699,2.034230,0.010567,78.0,3272.0,4936.0,0.0


In [36]:
result.nlargest(5, 'Trading_times')

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_20 MA_30,-0.129045,0.012500,0.009751,218.0,2917.0,4936.0,0.0
MA_21 MA_31,-0.021035,0.116042,0.009858,204.0,2923.0,4936.0,0.0
MA_22 MA_32,-0.001489,0.038751,0.009854,204.0,2937.0,4936.0,0.0
MA_20 MA_31,-0.158229,0.155261,0.009755,202.0,2935.0,4936.0,0.0
MA_21 MA_32,0.055279,0.196022,0.009934,198.0,2953.0,4936.0,0.0


In [37]:
result.nlargest(5, 'Holding_days')

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_90 MA_100,0.429737,1.143083,0.010355,78.0,3297.0,4936.0,0.0
MA_89 MA_99,0.400712,1.100510,0.010354,78.0,3295.0,4936.0,0.0
MA_88 MA_98,0.564257,1.146417,0.010458,80.0,3291.0,4936.0,0.0
MA_89 MA_100,0.425225,1.083729,0.010353,76.0,3291.0,4936.0,0.0
MA_88 MA_99,0.560491,1.060843,0.010458,74.0,3290.0,4936.0,0.0


In [38]:
result.nsmallest(5, 'Holding_days')

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_20 MA_30,-0.129045,0.012500,0.009751,218.0,2917.0,4936.0,0.0
MA_21 MA_31,-0.021035,0.116042,0.009858,204.0,2923.0,4936.0,0.0
MA_20 MA_31,-0.158229,0.155261,0.009755,202.0,2935.0,4936.0,0.0
MA_22 MA_32,-0.001489,0.038751,0.009854,204.0,2937.0,4936.0,0.0
MA_23 MA_33,0.158139,0.163592,0.010055,190.0,2951.0,4936.0,0.0


In [39]:
result.nsmallest(5, 'STD')

,Sharpe,Simple_rtn,STD,Trading_times,Holding_days,Total_days,Comm
Pairs,,,,,,,
MA_20 MA_30,-0.129045,0.012500,0.009751,218.0,2917.0,4936.0,0.0
MA_20 MA_31,-0.158229,0.155261,0.009755,202.0,2935.0,4936.0,0.0
MA_26 MA_36,0.344125,0.772667,0.009808,176.0,2966.0,4936.0,0.0
MA_22 MA_32,-0.001489,0.038751,0.009854,204.0,2937.0,4936.0,0.0
MA_21 MA_31,-0.021035,0.116042,0.009858,204.0,2923.0,4936.0,0.0


## Critiques

In [40]:
print("Asset simple return: ", df_ko['Close'][-1]/df_ko['Close'][0]-1)

Asset simple return:  4.089133911098507


In [41]:
df_ko.loc[df_ko.index, ['Daily_rtn']] = np.log(df_ko['Close']/df_ko['Close'].shift(1))

In [42]:
# twailing 250 days' KO's sharpe ratio
print("CocaCola 250Days Sharpe Ratio: ", 
      ((df_ko['Close'][-1]/df_ko['Close'][-251]-1)/np.sqrt(250))/df_ko['Daily_rtn'].tail(250).std() )

CocaCola 250Days Sharpe Ratio:  0.52938006829805


In [43]:
df_ko['Close'][-1]/df_ko['Close'][-251]

1.1044168869094089